<a href="https://colab.research.google.com/github/emamarela/ActiveLearning/blob/main/DistilRoBERT_MultiEurlex_DE_KMeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/webis-de/small-text

Cloning into 'small-text'...
remote: Enumerating objects: 2025, done.
remote: Counting objects: 100% (2025/2025), done.
remote: Compressing objects: 100% (1342/1342), done.
remote: Total 2025 (delta 1317), reused 1377 (delta 669), pack-reused 0
Receiving objects: 100% (2025/2025), 400.73 KiB | 11.45 MiB/s, done.
Resolving deltas: 100% (1317/1317), done.


In [ ]:
!pip install transformers
!pip install numpy==1.20.0
!pip install datasets

In [ ]:
import sys
sys.path.insert(0, '/content/small-text/')
sys.path.insert(0, '/content/small-text/examples')


In [ ]:
import logging

import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer

from small_text.active_learner import PoolBasedActiveLearner
from small_text.initialization import random_initialization_stratified
from small_text.integrations.transformers import TransformerModelArguments
from small_text.integrations.transformers.classifiers.factories import TransformerBasedClassificationFactory
from small_text.query_strategies import PoolExhaustedException, EmptyPoolException
from small_text.query_strategies import RandomSampling, LeastConfidence, EmbeddingKMeans

from examplecode.data.example_data_multilabel import get_train_test
from examplecode.data.example_data_transformers import preprocess_data
from examplecode.shared import evaluate_multi_label

TRANSFORMER_MODEL = TransformerModelArguments('distilbert-base-german-cased')  #distilbert-base-german-cased

In [ ]:
TRANSFORMER_MODEL = TransformerModelArguments('distilroberta-base')

num_classes = 21
clf_factory = TransformerBasedClassificationFactory(TRANSFORMER_MODEL,
                                                    num_classes,
                                                    kwargs=dict({
                                                        'device': 'cuda',
                                                        'multi_label': True
                                                    }))
query_strategy = EmbeddingKMeans()

tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL.model, cache_dir='.cache/')

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
results = []

def perform_active_learning(active_learner, train, labeled_indices, test):
    # Perform 10 iterations of active learning...
    for i in range(10):
        # ...where each iteration consists of labelling 20 samples
        q_indices = active_learner.query(num_samples=20)

        # Simulate user interaction here. Replace this for real-world usage.
        y = train.y[q_indices]

        # Return the labels for the current query to the active learner.
        active_learner.update(y)

        labeled_indices = np.concatenate([q_indices, labeled_indices])

        print('Iteration #{:d} ({} samples)'.format(i, len(labeled_indices)))
        evaluate_multi_label(active_learner, train[labeled_indices], test)
        results.append(evaluate_multi_label(active_learner, train[labeled_indices], test))


In [ ]:
def initialize_active_learner(active_learner, y_train):

    x_indices_initial = random_initialization_stratified(y_train, n_samples=400)
    y_initial = y_train[x_indices_initial]

    active_learner.initialize_data(x_indices_initial, y_initial)

    return x_indices_initial

In [ ]:
eurlex_de = load_dataset('multi_eurlex', 'de')

Reusing dataset multi_eurlex (/root/.cache/huggingface/datasets/multi_eurlex/de/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def main():
    # Active learning parameters
    num_classes = 21
    clf_factory = TransformerBasedClassificationFactory(TRANSFORMER_MODEL,
                                                        num_classes,
                                                        kwargs=dict({
                                                            'device': 'cuda',
                                                            'multi_label': True
                                                        }))
    query_strategy = EmbeddingKMeans()

    # Prepare some data
    #train, test = get_train_test()
    train = eurlex_de['train']
    test = eurlex_de['validation']
    tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL.model, cache_dir='.cache/')
    x_train = preprocess_data(tokenizer, train['text'], train['labels'], multi_label=True)
    x_test = preprocess_data(tokenizer, test['text'], test['labels'], multi_label=True)
    # Active learner
    active_learner = PoolBasedActiveLearner(clf_factory, query_strategy, x_train)
    
    
    labeled_indices = initialize_active_learner(active_learner, x_train.y)
    print(6)

    try:
        perform_active_learning(active_learner, x_train, labeled_indices, x_test)
    except PoolExhaustedException:
        print('Error! Not enough samples left to handle the query.')
    except EmptyPoolException:
        print('Error! No more samples left. (Unlabeled pool is empty)')

In [ ]:
if __name__ == '__main__':
    logging.getLogger('small_text').setLevel(logging.INFO)

    main()

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

...

Iteration #7 (560 samples)
Train accuracy: 0.53
Test accuracy: 0.40
---
Train accuracy: 0.53
Test accuracy: 0.40
---


100%|██████████| 55000/55000 [08:38<00:00, 106.06it/s]
Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-bas

Iteration #8 (580 samples)
Train accuracy: 0.52
Test accuracy: 0.40
---
Train accuracy: 0.52
Test accuracy: 0.40
---


100%|██████████| 55000/55000 [08:38<00:00, 106.05it/s]
Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-bas

Iteration #9 (600 samples)
Train accuracy: 0.56
Test accuracy: 0.31
---
Train accuracy: 0.56
Test accuracy: 0.31
---
